# 05 -  Attributuion Modelling for Online Clickstream Data

# Introduction

## What is a Markov Chain?

A Markov chain is a type of probabilistic model. This means that it is a system for representing different states that are connected to each other by probabilities.

The state, in the example of our attribution model, is the channel or tactic that a given user is exposed to (e.g. a nonbrand SEM ad or a Display ad). The question then becomes, given your current state, what is your next most likely state?

Well one way to estimate this would be to get a list of all possible states branching from the state in question and create a conditional probability distribution representing the likelihood of moving from the initial state to each other possible state.

## Attributing the Conversions

Now that we have constructed the system that represents our user behavior it's time to use it to re-allocate the total number of conversions that occurred for a period of time.

What I like to do is take the entire system's probability matrix and simulate thousands of runs through the system that end when our simulated user arrives at either conversion or null. This allows us to use a rather small sample to generalize because we can simulate the random walk through the different stages of our system with our prior understanding of the probability of moving from one stage to the next. Since we pass a probability distribution into the mix we are allowing for a bit more variation in our simulation outcomes.

After getting the conversion rates of the system we can simulate what occurs when we remove channels from the system one by one to understand their overall contribution to the whole.

We do this by calculating the removal effect1 which is defined as the percentage of conversions we'd miss out on if a given channel or tactic was removed from the system.

In other words, if we create one new model for each channel where that channel is set to 100% no conversion, we will have a new model that highlights the effect that removing that channel entirely had on the overall system.

Mathematically speaking, we'd be taking the percent difference in the conversion rate of the overall system with a given channel set to NULL against the conversion rate of the overall system. We would do this for each channel. Then we create a weighting for each of them based off of the sum of removal effects and then we could finally then multiply that number by the number of conversions to arrive at the fractionally attributed number of conversions.

If the above paragraph confuses you head over to here and scroll about a third of the way down for a clear removal effect example. I went and made my example system too complicated for me to want to manually write out the the removal effect CVRs.

In [3]:
import pandas as pd

In [15]:
data = pd.read_csv('../data/raw/0.tsv', sep='\t', header=None)

In [16]:
data

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,1331799426,2012-03-15 01:17:06,2860005755985467733,4611687631188657821,FAS-2.8-AS3,N,0,99.122.210.248,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1331800486,2012-03-15 01:34:46,2859997896193943381,6917530184062522013,FAS-2.8-AS3,N,0,69.76.12.213,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1331857433,2012-03-15 17:23:53,2781404195155152050,6917530222178992983,FAS-2.8-AS3,N,0,67.240.15.94,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1331856300,2012-03-15 17:05:00,2781404195155152050,6917530222178992983,FAS-2.8-AS3,N,0,67.240.15.94,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1331800073,2012-03-15 01:27:53,2829533710344657576,6917530841727896174,FAS-2.8-AS3,N,0,98.234.107.75,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
5,1331802578,2012-03-15 02:09:38,5279782221568288854,732692051711230006,FAS-2.8-AS3,N,0,75.85.165.38,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
6,1331831882,2012-03-15 10:18:02,2754911479663756329,6917530150775072471,FAS-2.8-AS3,N,0,71.53.206.175,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
7,1331836722,2012-03-15 11:38:42,2838741409408826824,6917530129301952401,FAS-2.8-AS3,N,0,97.96.62.161,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
8,1331840579,2012-03-15 12:42:59,2829471613707461802,4611687624208293250,FAS-2.8-AS3,N,0,129.119.158.240,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
9,1331834569,2012-03-15 11:02:49,2842838286372024450,6917530630737670784,FAS-2.8-AS3,N,0,96.241.99.50,1,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
from markovclick.models import MarkovClickstream
m = MarkovClickstream(data)
fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
sns.heatmap(m.prob_matrix, xticklabels=m.pages, yticklabels=m.pages,cmap="YlGnBu")